# Data Science in the Wild Project Phase 1
**Team Members:**
- Hongyiming Cui
- Xiaohui Zang
- Jiawei Wang
- George Zhu

## GitHub Repository Link: 

https://github.com/MarinaCui/Data-Science-in-the-Wild-Final-Project

## Project and Research Questions Overview
This research aims to investigate the evolving intersection of economic health and physical well-being in the United States during a critical period of global recovery. Using a state-level panel dataset spanning 2021 to 2024, I will analyze how various dimensions of income—including absolute levels, distribution patterns, and household compositions—correlate with high-impact health outcomes such as heart disease and diabetes mortality, obesity rates, and self-reported health status.

## Research Questions

- To what extent does the "Socioeconomic Gradient" explain variations in chronic disease mortality across the United States?

- How has the relationship between economic prosperity and public health shifted during the post-pandemic recovery period (2021–2024)?

- Does household structure act as a buffer or a risk factor in the relationship between income and chronic disease?

- Is public health more sensitive to the average wealth of a state or the equitable distribution of that wealth?

## Raw Data Description

### Income Data

* Source: U.S. Census Bureau, American Community Survey (ACS) 1-Year Estimates (Table S1901: Income in the Past 12 Months).

    - https://www.census.gov/programs-surveys/acs 

* Timeframe: 2021, 2022, 2023, and 2024.

* Granularity: State-level (50 U.S. states, the District of Columbia, and Puerto Rico).

* Key Variables:

    - Household Income: Median and mean income for all households.

    - Family Structure Income: Specific median/mean income for "Married-couple families" versus "Non-family households."

    - Income Distribution: Percentage of households within specific income brackets (ranging from <$10k to >$200k).

### Health Status and Behavioral Risk Data

* Source: Kaiser Family Foundation (KFF) State Health Facts, derived from the CDC’s Behavioral Risk Factor Surveillance System (BRFSS).

    - https://www.kff.org/state-category/health-status/

* Timeframe: 2021–2024.

* Key Variables:

    - Self-Reported Health Status: The percentage of adults reporting their health as "Poor," "Fair," "Good," "Very Good," or "Excellent."

    - Obesity Metrics: Prevalence of obesity (BMI 30–39.9) and severe obesity (BMI 40+).


### Chronic Disease Mortality Data

- Source: Kaiser Family Foundation (KFF) State Health Facts, aggregating data from the CDC WONDER (National Center for Health Statistics).

    - https://www.kff.org/state-category/health-status/

- Key Variables:

    - Heart Disease Deaths: Total deaths and age-adjusted mortality rates per 100,000 population.

    - Diabetes Deaths: Total deaths and age-adjusted mortality rates per 100,000 population.

## Variable Construction

- **Diabetes:** Death rate per 100,000 population.
- **Heart Disease:** Death rate per 100,000 population.
- **Obesity:** Defined as BMI ≥ 30. This was calculated as the sum of:
  - Obese (BMI 30–39.9)
  - Severely Obese (BMI ≥ 40)
- **Poor Health:** Percentage of adults reporting poor health.

All variables are state-level annual indicators.

## Data Cleaning

In [14]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns      

### Health Data

In [15]:
years = [2021, 2022, 2023, 2024]

valid_states = [
'Alabama','Alaska','Arizona','Arkansas','California','Colorado',
'Connecticut','Delaware','District of Columbia','Florida','Georgia',
'Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky',
'Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota',
'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire',
'New Jersey','New Mexico','New York','North Carolina','North Dakota',
'Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island',
'South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont',
'Virginia','Washington','West Virginia','Wisconsin','Wyoming'
]

In [16]:
def clean_obesity(file_path, new_name):
    df = pd.read_csv(file_path, skiprows=2)
    df.columns = df.columns.str.strip()

    df = df[[
        "Location",
        "Obese (BMI 30-39.9)",
        "Severely Obese (BMI of 40 or Higher)"
    ]]

    df = df[df["Location"].isin(valid_states)]

    df["Obese (BMI 30-39.9)"] = pd.to_numeric(
        df["Obese (BMI 30-39.9)"], errors="coerce"
    )
    df["Severely Obese (BMI of 40 or Higher)"] = pd.to_numeric(
        df["Severely Obese (BMI of 40 or Higher)"], errors="coerce"
    )

    df[new_name] = (
        df["Obese (BMI 30-39.9)"] +
        df["Severely Obese (BMI of 40 or Higher)"]
    )

    df = df[["Location", new_name]]
    df.columns = ["State", new_name]

    return df

In [17]:
def clean_file(file_path, value_column, new_name):
    df = pd.read_csv(file_path, skiprows=2)
    df.columns = df.columns.str.strip()
    df = df[["Location", value_column]]
    df = df[df["Location"].isin(valid_states)]
    df.columns = ["State", new_name]
    df[new_name] = pd.to_numeric(df[new_name], errors="coerce")
    return df

final_df = None

health_data_folder = "health_data/" 
final_df = None

for year in years:
    
    d = clean_file(
        f"{health_data_folder}Diabetes_{year}.csv", 
        "Diabetes Death Rate per 100,000 Population",
        f"Diabetes_{year}"
    )
    
    h = clean_file(
        f"{health_data_folder}Heart_Disease_{year}.csv", 
        "Heart Disease Death Rate per 100,000",
        f"Heart_Disease_{year}"
    )
    
    o = clean_obesity(
        f"{health_data_folder}Obesity_{year}.csv", 
        f"Obesity_{year}"
    )
    
    s = clean_file(
        f"{health_data_folder}Health_Status_{year}.csv", 
        "Poor",
        f"Poor_Health_{year}"
    )
    
    year_df = d.merge(h, on="State") \
               .merge(o, on="State") \
               .merge(s, on="State")
    
    if final_df is None:
        final_df = year_df
    else:
        final_df = final_df.merge(year_df, on="State")

# -------- Multi-level column formatting --------

state_col = final_df["State"]
data_cols = final_df.drop(columns=["State"])

new_columns = []
for col in data_cols.columns:
    disease, year = col.rsplit("_", 1)
    new_columns.append((year, disease))

data_cols.columns = pd.MultiIndex.from_tuples(new_columns)

final_df_multi = data_cols.copy()
final_df_multi.index = state_col
final_df_multi = final_df_multi.sort_index(axis=1, level=0)

final_df_multi = final_df_multi.round(3)

print(final_df_multi)

final_df_multi.to_csv("final_health_dataset.csv")

                         2021                                       2022  \
                     Diabetes Heart_Disease Obesity Poor_Health Diabetes   
State                                                                      
Alabama                  26.3         247.5   0.389       0.057     25.4   
Alaska                   27.0         154.7   0.338       0.034     25.6   
Arizona                  27.3         158.3   0.311       0.040     25.1   
Arkansas                 39.3         231.0   0.382       0.069     36.0   
California               25.5         147.8   0.283       0.040     25.0   
Colorado                 19.0         135.1   0.253       0.032     18.5   
Connecticut              15.9         136.7   0.297       0.027     15.2   
Delaware                 23.9         162.7   0.352       0.034     25.1   
District of Columbia     24.4         192.8   0.242       0.029     18.2   
Florida                  24.8         151.3     NaN         NaN     22.3   
Georgia     

### Income Data

In [18]:
import pandas as pd
import glob
import os

BASE_DIR = os.getcwd()  # current working directory
folder = os.path.join(BASE_DIR, "income_data")

files = sorted(glob.glob(os.path.join(folder, "*.csv")))

In [19]:
# merge data in different years and expand to long-data format
all_parts = []
for f in files:
    year = os.path.basename(f)[7:11]
    df = pd.read_csv(f)
    long = df.melt(id_vars=['Label (Grouping)'], var_name='State_Measure', value_name='Value')
    def split_cat(x):
        pieces = x.split('!!')
        state = pieces[0]
        category = '!!'.join(pieces[1:]) if len(pieces) > 1 else ''
        return pd.Series([state, category])
    long[['State','Category']] = long['State_Measure'].apply(split_cat)
    long['Year'] = year
    all_parts.append(long[['Year','State','Category','Label (Grouping)','Value']])
full = pd.concat(all_parts, ignore_index=True)
full.to_csv('income_long_all.csv', index=False)
print('long form saved to income_long_all.csv (shape', full.shape,')')

# transform to wide data
long = full  
mask = ~long['Label (Grouping)'].str.contains(r"\$|Less|to|more", regex=True)
wide_df = long[mask].copy()
wide_df['Year'] = wide_df['Year'].astype(str)
wide_df['col'] = wide_df['Year'] + '_' + wide_df['Category'] + '_' + wide_df['Label (Grouping)']
wide = wide_df.pivot(index='State', columns='col', values='Value')
print('example columns:', wide.columns[:10])
print('wide shape:', wide.shape)
wide.reset_index().to_csv('state_year_variables.csv', index=False)
print('wide file saved to state_year_variables.csv (shape', wide.shape,')')

long form saved to income_long_all.csv (shape (14144, 5) )
example columns: Index(['2021_Families!!Estimate_Mean income (dollars)',
       '2021_Families!!Estimate_Median income (dollars)',
       '2021_Families!!Estimate_PERCENT ALLOCATED',
       '2021_Families!!Estimate_Total',
       '2021_Families!!Estimate_    Family income in the past 12 months',
       '2021_Families!!Estimate_    Household income in the past 12 months',
       '2021_Families!!Estimate_    Nonfamily income in the past 12 months',
       '2021_Households!!Estimate_Mean income (dollars)',
       '2021_Households!!Estimate_Median income (dollars)',
       '2021_Households!!Estimate_PERCENT ALLOCATED'],
      dtype='object', name='col')
wide shape: (52, 112)
wide file saved to state_year_variables.csv (shape (52, 112) )


In [20]:
# remove all-nan columns
wide_clean = wide.dropna(axis=1, how='all')
print('removed all-NaN cols, new shape', wide_clean.shape)

# delete columns with high nan-rate
threshold = 0.5
na_frac = wide_clean.isna().mean()
high_na_cols = na_frac[na_frac > threshold].index.tolist()
print('columns with >50% NA:', high_na_cols)
wide_clean = wide_clean.drop(columns=high_na_cols)
print('after dropping high-NA cols', wide_clean.shape)

# delete verbose variables
cols_all_x = [c for c in wide_clean.columns if (wide_clean[c] == '(X)').all()]
wide_clean = wide_clean.drop(columns=cols_all_x)

# fillna
wide_clean = wide_clean.fillna('N/A')
wide_clean.reset_index().to_csv('final.csv', index=False)

removed all-NaN cols, new shape (52, 96)
columns with >50% NA: []
after dropping high-NA cols (52, 96)


### Merge two dataset together

In [21]:
income_flat = pd.read_csv("final.csv", index_col="State")

# build multiindex: split on first underscore (year, rest-of-name)
newcols = []
for c in income_flat.columns:
    pieces = c.split("_", 1)
    if len(pieces) == 2:
        year, var = pieces
    else:
        year, var = pieces[0], ""
    newcols.append((year, var))

income_flat.columns = pd.MultiIndex.from_tuples(newcols)
income_wide = income_flat.sort_index(axis=1, level=0)

# read health file with two header rows
health_temp = pd.read_csv("final_health_dataset.csv", header=[0,1])
first_col = health_temp.columns[0]
health_wide = health_temp.set_index(first_col)
# rename index
health_wide.index.name = 'State'

combined = pd.concat([income_wide, health_wide], axis=1)
combined = combined.sort_index(axis=1, level=0)

print("combined shape", combined.shape)
combined.to_csv("final_all_dataset.csv")
print("combined file written to final_all_dataset.csv")

combined shape (53, 76)
combined file written to final_all_dataset.csv


In [22]:
# See how many missing values
final_df_multi.isna().sum()

2021  Diabetes         0
      Heart_Disease    0
      Obesity          1
      Poor_Health      1
2022  Diabetes         0
      Heart_Disease    0
      Obesity          0
      Poor_Health      0
2023  Diabetes         0
      Heart_Disease    0
      Obesity          2
      Poor_Health      2
2024  Diabetes         0
      Heart_Disease    0
      Obesity          1
      Poor_Health      1
dtype: int64

## Plain English Explanation of the Data Processing Code

### Builds a Health Dataset (2021–2024)

- Reads yearly CSV files for:
  - Diabetes  
  - Heart Disease  
  - Obesity  
  - Poor Health  

- Cleans the data:
  - Keeps only valid U.S. states  
  - Converts values to numeric  
  - Combines moderate + severe obesity into a single obesity measure  

- Merges all metrics by **state and year**.
- Restructures columns into a **multi-level format (Year → Metric)**.
- Saves the cleaned health dataset.

### Builds an Income Dataset

- Reads all income CSV files from a specified folder.
- Converts each file from **wide → long format**.
- Extracts:
  - State  
  - Category  
  - Year  

- Combines all years into one dataset.
- Converts the dataset back to **wide format** (one column per year + variable).
- Cleans the data by:
  - Removing fully missing columns  
  - Dropping columns with >50% missing values  
  - Removing invalid placeholder values (“(X)”)  
  - Filling remaining missing values with `"N/A"`  

- Saves the cleaned income dataset.

### Combines Health and Income Data

- Reformats both datasets to use **multi-level columns by year**.
- Merges them by state.
- Outputs a final combined dataset:  

## Description of Final Dataset

The final dataset (`final_all_dataset.csv`) is a **state-level, multi-year panel dataset** combining health and income variables.

### Structure

- **Rows:**  
  - One row per U.S. state (50 states + District of Columbia).  
  - Each row represents a single state.

- **Columns:**  
  - Multi-level (hierarchical) columns:
    - **Level 1:** Year (2021–2024)  
    - **Level 2:** Variable name  

- Variables include:
  - **Health metrics:**
    - Diabetes death rate (per 100,000)
    - Heart disease death rate (per 100,000)
    - Obesity rate (combined moderate + severe)
    - Poor health percentage
  - **Income-related variables:**
    - Multiple income categories and measures derived from the Census data  
    - Cleaned to remove invalid or high-missing columns


## Limitations

1. The dataset is aggregated at the state level and does not capture within-state variation.
2. Some variables are based on self-reported survey data (BRFSS), which may introduce reporting bias.
3. The time period (2021–2024) is relatively short for long-term trend analysis.
4. A small number of missing observations exist in obesity and poor health indicators.

## Questions for TA

1. Is a four-year panel sufficient for regression analysis in this context?
2. Would it be preferable to reshape the dataset into long format instead of wide multi-level format?
3. Should additional socioeconomic control variables be added at this stage?